In [ ]:
!pip install PyPDF2

In [ ]:
!pip install ace_tools

In [ ]:
!pip3 install pdfminer.six

In [ ]:
!pip install wordfreq

In [24]:
!pip install --upgrade pymorphy2


In [33]:
import pandas as pd
import re
from pdfminer.high_level import extract_text
from wordfreq import word_frequency

file_path = "/content/Володин и др._2021_Полный ительменско-русский словарь (1)-397-488.pdf"
text = extract_text(file_path)

# Список тегов
pos_tags = {"сущ", "мест", "п1", "п2", "неп", "нареч", "прил", "межд", "собств", "инф", "союз"}

# Игнорируемые пометы (например, с. ю.)
ignore_tags = {"с.", "ю."}

itelmen_markers = re.compile(r"[ʼӄӈӽӼӃӇʔ˚ӑӘәЉљԒԓŎŏЊњ]")  # Ительменские символы
itelmen_consonant_e = re.compile(r"[бвгджзклмнпрстфхцчшщ]э")  # Согласная + э

# Разбиваем текст на строки
lines = text.split("\n")

data = []

def is_russian(word):
    """Проверяет, является ли слово русским."""
    if word_frequency(word, "ru") > 0:
        return True
    if re.match(r"^[а-яА-ЯёЁ-]+$", word):
        if is_itelmen(word):
            return False
        return True
    return False

def is_itelmen(word):
    """Проверяет, является ли слово ительменским."""
    return bool(itelmen_markers.search(word) or itelmen_consonant_e.search(word))

def process_entry(entry):
    """Обрабатывает запись: разделяет русские слова, ительменские слова и части речи."""
    if not entry:
        return

    russian_part = []
    itelmen_part = []
    pos_part = []
    word_to_pos = {}  # Словарь "слово → тег"

    i = 0
    while i < len(entry):
        word = entry[i]

        # Игнорируем ненужные пометы (с. ю.)
        if word in ignore_tags:
            i += 1
            continue

        # Если следующее слово - тег (сущ, прил и т. д.), привязываем его к текущему слову
        if i + 1 < len(entry) and entry[i + 1] in pos_tags:
            word_to_pos[word] = entry[i + 1]  # Запоминаем тег для слова
            i += 1  # Пропускаем тег

        # Если слово - русское
        if is_russian(word):
            russian_part.append(word)

        # Если слово - ительменское
        elif is_itelmen(word):
            itelmen_part.append(word)

        i += 1

    # Проставляем теги для ительменских слов
    itelmen_part_cleaned = []
    pos_part_cleaned = []

    for word in itelmen_part:
        itelmen_part_cleaned.append(word)
        if word in word_to_pos:
            pos_part_cleaned.append(word_to_pos[word])  # Добавляем тег

    # Сохраняем данные
    if russian_part and itelmen_part_cleaned:
        data.append([" ".join(russian_part), "; ".join(itelmen_part_cleaned), ", ".join(pos_part_cleaned) if pos_part_cleaned else ""])

# Обрабатываем строки
current_entry = []
for line in lines:
    words = line.split()

    if not words:
        continue  # Пропускаем пустые строки

    # Если новая строка начинается с русского слова, обрабатываем предыдущую запись
    if is_russian(words[0]):
        process_entry(current_entry)
        current_entry = words  # Начинаем новую запись
    else:
        # Если строка продолжает перевод, добавляем слова к текущей записи
        current_entry.extend(words)

# Обрабатываем последнюю запись
process_entry(current_entry)

# Создаем DataFrame
df = pd.DataFrame(data, columns=["Русское слово", "Ительменское слово", "Часть речи"])

# Выводим результат
from IPython.display import display
display(df)

df.to_csv("russian_itelmen_dictionary_cleaned4.csv", index=False)


,Русское слово,Ительменское слово,Часть речи
0,а,синэх,союз
1,а ну-ка,хайʔэ,межд
2,автомобиль,ԓалхтэра,сущ
3,ад,вэʔэнна,сущ
4,акиба фитвит сущ;,фичскʼэ,сущ
...,...,...,...
4961,ярко светить,азлаʔԓ-,неп
4962,яростный,чинзлаӽ,прил
4963,ясли,нукнэн; ˚ӄэпх,сущ
4964,ястреб сущ; пхакч,ипәлакʼич,


In [34]:
from google.colab import files
files.download("russian_itelmen_dictionary_cleaned4.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>